In [62]:
from os import listdir
import jsonlines
import itertools
import nltk
from nltk.corpus import stopwords
import pymorphy2 as pm
import networkx as nx
import re
from bisect import bisect_left
#from joblib import Parallel, delayed
from collections import Counter
from tqdm import trange
from tqdm import tqdm
from math import log
from random import shuffle
from sys import exit
import numpy as np
import json
import codecs
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.neighbors import NearestCentroid
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from scipy.sparse import load_npz
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from scipy.sparse import vstack

from sklearn.externals import joblib
#nltk.download()
tknzr = nltk.TweetTokenizer()

In [2]:
ok_set = set()
with open("sources/accepted_categories.txt", mode="r", encoding="utf-8") as inp:
    for line in inp:
        line = line[:-1]
        ok_set.add(line)

ok = sorted(ok_set)

categories_dict = {}
with open("sources/article_cat.json", mode="r") as input:
    categories_dict = json.loads(input.read())
    
category_article = {}
with open("sources/cat_article.json", mode="r") as input:
    category_article = json.loads(input.read())
    
X = load_npz("sources/text_tfidf.npz")
    
texts = {item['id']: item['text'] for item in jsonlines.open('sources/normalized_texts.jl', 'r')}

In [3]:
ids = sorted([id for id in texts.keys()])

In [4]:
vectorizer = TfidfVectorizer(min_df=5, max_df=0.98)
text_tfidf = vectorizer.fit_transform([text for (id, text) in sorted(texts.items())])
#svd = TruncatedSVD(n_components=500, random_state=27)
#X_svd = svd.fit_transform(text_tfidf)
X = text_tfidf

In [4]:
k_nearest = 5
'''class_centroids = {}
with open("sources/centroids.json", "r") as input:
    class_centroids = json.loads(input.read())
'''
class_centroids = load_npz("sources/sparce_centroids_nosvd.npz")

kn_clust = NearestNeighbors(n_neighbors=k_nearest)
#kn_clust.fit([centroid for (cat, centroid) in sorted(class_centroids.items())])
kn_clust.fit(class_centroids)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [5]:
class_centroids.shape

(14697, 121303)

In [6]:
X.shape

(96794, 121303)

In [46]:
n_splits = 3
kf = KFold(n_splits=n_splits, shuffle=True, random_state=27)
clf = LogisticRegression(C=1.0, class_weight="balanced")
X_train = []
X_right = []
X_left = []
y_train = []
rows = []

train_index, test_index = 0, 0
for item in kf.split(ids):
    
    score = 0
    train_index, test_index = item[0], item[1]
    
    print("TRAIN:", train_index, "TEST:", test_index)
    print(len(train_index), len(test_index))
    
    X_left[:] = []
    for i in trange(len(train_index)):
        for j in range(k_nearest):
            X_left.append(X[train_index[i]])
            
    X_left = vstack(X_left)
    X_right[:] = []
    y_train[:] = []
    rows[:] = []
    print("Train!")
    batchsize = 1000
    for i in trange(0, len(train_index), batchsize):
        batch = train_index[i:i+batchsize]
        curr_x = X[batch]
        _neighbors = kn_clust.kneighbors(curr_x, n_neighbors=k_nearest, return_distance=False)
        
        for j, neighbors in enumerate(_neighbors):  
            cats = categories_dict[ids[i+j]]
            neighbors = [ok[c] for c in neighbors]
            common = set(neighbors).intersection(set(cats))
            for neigh in neighbors:
                rows.append(ok.index(neigh))
                y_train.append(1 if neigh in common else 0)
    
    X_right = X[rows]
    print(X_right.shape)
    print(X_left.shape)
    X_train = hstack([X_left, X_right])
    clf.fit(X_train, y_train)
    
    print("Test!")
    for index in tqdm(test_index):
        result = ids[index]
        test_svd = X[index]
        y_true = np.array(categories_dict[result])
        X_local = []
        local_skipped = []
        prediction = []
        rows[:] = []
        for i, y in enumerate(y_true):
            if ok.index(y) is not None:
                rows.append(ok.index(y))
            else:
                local_skipped.append((i, y))
                
        test_svd = vstack([test_svd] * class_centroids[rows].shape[0])
        X_local = hstack([test_svd, class_centroids[rows]])
        if X_local.shape[0] > 0:
            prediction = clf.predict(X_local)
        for item in local_skipped:
            prediction.insert(item[0], 1)
            
        score += sum(prediction) / len(y_true)
    
    print(score / len(test_index))   
    break

TRAIN: [    0     1     2 ... 96791 96792 96793] TEST: [    4     7    27 ... 96770 96779 96789]
77435 19359


100%|██████████████████████████████████████████████████████████████████████████| 77435/77435 [00:27<00:00, 2835.72it/s]


Train!


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [03:07<00:00,  2.40s/it]


(387175, 121303)
[0. 0. 0. ... 0. 0. 0.]


KeyboardInterrupt: 

In [57]:
sum(y_train)

1997

In [58]:
X_train.shape

(387175, 242606)

In [59]:
clf = LogisticRegression(C=1.0, class_weight="balanced")

In [60]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [70]:
for index in tqdm(test_index):
    result = ids[index]
    test_svd = X[index]
    y_true = np.array(categories_dict[result])
    X_local = []
    local_skipped = []
    prediction = []
    rows[:] = []
    for i, y in enumerate(y_true):
        if ok.index(y) is not None:
            rows.append(ok.index(y))
        else:
            local_skipped.append((i, y))
    
    test_svd = vstack([test_svd] * class_centroids[rows].shape[0])
    X_local = hstack([test_svd, class_centroids[rows]])
    if X_local.shape[0] > 0:
        prediction = clf.predict_proba(X_local)[:,1]
    for item in local_skipped:
        prediction.insert(item[0], 1)

    score += sum(prediction) / len(y_true)
    print(prediction)
    print(roc_auc_score([1] * len(y_true), prediction))
    
    break
#print(score / len(test_index))   

  0%|                                                                                        | 0/19359 [00:00<?, ?it/s]

[0.00088934 0.00091905]


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

### Обычный cross_val

In [67]:
X_left = []
for x in tqdm(X):
    for j in range(k_nearest):
        X_left.append(x)       
X_left = vstack(X_left)

X_right = []
y_train[:] = []
rows[:] = []
batchsize = 1000
for i in trange(0, X.shape[0], batchsize):
    curr_x = X[i:i+batchsize]
    _neighbors = kn_clust.kneighbors(curr_x, n_neighbors=k_nearest, return_distance=False)
    
    for j, neighbors in enumerate(_neighbors):
        cats = categories_dict[ids[i+j]]
        neighbors = [ok[c] for c in neighbors]
        common = set(neighbors).intersection(set(cats))
        for neigh in neighbors:
            rows.append(ok.index(neigh))
            y_train.append(1 if neigh in common else 0)
            

X_right = X[rows]
X_full = hstack([X_left, X_right])

96794it [00:04, 23562.51it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 97/97 [04:02<00:00,  2.50s/it]


In [16]:
X_full.shape

(483970, 242606)

In [17]:
sum(y_train)

192465

In [89]:
cross_val_score(clf, X_full, y_train, n_jobs=-1, scoring="f1")

array([0.65530246, 0.62893164, 0.59896771])

### Обучаем модельку

In [30]:
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=27)
clf = LogisticRegression(C=1.0, class_weight="balanced")

train_index, test_index = 0, 0
for item in kf.split(ids):
    
    score = 0
    train_index, test_index = item[0], item[1]
    
    print("TRAIN:", train_index, "TEST:", test_index)
    print(len(train_index), len(test_index))
    break
    
    
X_left = []
for x in tqdm(X):
    for j in range(k_nearest):
        X_left.append(x)       
X_left = vstack(X_left)

X_right = []
y = []
rows = []
batchsize = 1000
for i in trange(0, X.shape[0], batchsize):
    curr_x = X[i:i+batchsize]
    _neighbors = kn_clust.kneighbors(curr_x, n_neighbors=k_nearest, return_distance=False)
    
    for j, neighbors in enumerate(_neighbors):
        cats = categories_dict[ids[i+j]]
        neighbors = [ok[c] for c in neighbors]
        common = set(neighbors).intersection(set(cats))
        for neigh in neighbors:
            rows.append(ok.index(neigh))
            y.append(1 if neigh in common else 0)

X_right = X[rows]
X_full = hstack([X_left, X_right])
y = np.array(y)

TRAIN: [    0     1     2 ... 96791 96792 96793] TEST: [    4     7    27 ... 96770 96779 96789]
77435 19359


96794it [00:03, 24582.64it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 97/97 [03:59<00:00,  2.47s/it]


In [60]:
X_full = X_full.tocsr()

In [61]:
clf.fit(X_full[train_index], y[train_index])

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [63]:
joblib.dump(clf, "sources/clf_binlogreg.pkl")

['sources/clf_binlogreg.pkl']